In [2]:
%pwd

'd:\\ds-projects\\Chicken-Disease-Classification\\research'

In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\ds-projects\\Chicken-Disease-Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
         
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [9]:
import importlib
import ChickenDiseaseClassifier.utils.common as common
importlib.reload(common)


<module 'ChickenDiseaseClassifier.utils.common' from 'd:\\ds-projects\\chicken-disease-classification\\src\\ChickenDiseaseClassifier\\utils\\common.py'>

In [10]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassifier import logger
from ChickenDiseaseClassifier.utils.common import get_size

In [11]:
import ChickenDiseaseClassifier.utils.common as common
print(dir(common))


['Any', 'BoxValueError', 'ConfigBox', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'base64', 'create_directories', 'decodeImage', 'encodeImageIntoBase64', 'ensure_annotations', 'get_size', 'joblib', 'json', 'load_bin', 'load_json', 'logger', 'os', 'read_yaml', 'save_bin', 'save_json', 'yaml']


In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2024-10-19 12:28:55,503 INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-19 12:28:55,516 INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-19 12:28:55,519 INFO: common: created directory at: artifacts]
[2024-10-19 12:28:55,521 INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-19 12:29:24,293 INFO: 1465807739: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11642226
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0a96177b8fd5905f9c9db1ebd966f8fe4ea7c8f72750825484f0b97150a9d216"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4456:2E4C79:1624F4:1C2264:671358B1
Accept-Ranges: bytes
Date: Sat, 19 Oct 2024 06:58:58 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100034-HYD
X-Cache: MIS